In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.environ.get('riot_api_key')


In [2]:
import pygsheets 
google_url = os.environ.get('google_url')
service_account = pygsheets.authorize(service_account_file = 'JSONS\\spreadsheet-automator-447604-0758cb3da318.json')
sheet = service_account.open_by_url(google_url)

In [3]:
import pandas as pd 
import requests
import time
def getLadder(rank,division):
    divisions = ['II','III','IV']
    page = pd.DataFrame()
    root = 'https://na1.api.riotgames.com/'
    meta = f'lol/league/v4/entries/RANKED_SOLO_5x5/{rank}/{division}?page=1'
    api_url = root + meta + '&api_key=' + api_key
    api_response = requests.get(api_url)
    api_df = pd.DataFrame(api_response.json())
    page = pd.concat([page,api_df])
    page_num = 2 
    while not api_df.empty:
        meta = f'lol/league/v4/entries/RANKED_SOLO_5x5/{rank}/{division}?page={page_num}'
        api_url = root + meta + '&api_key=' + api_key
        api_response = requests.get(api_url)
        api_df = pd.DataFrame(api_response.json())
        page = pd.concat([page,api_df])
        page_num+=1
        time.sleep(1)
    return page

In [4]:
def get_puuid(summoner_id):
    root = 'https://na1.api.riotgames.com/lol/summoner/v4/summoners/'
    api_url = root + summoner_id + '?api_key=' + api_key
    page = requests.get(api_url)
    return page.json()['puuid']

In [5]:
if os.path.exists('ironLadder.csv'):
    ironLadder = pd.read_csv('ironLadder.csv')
else:
    ironLadder = getLadder('IRON','I').sort_values('leaguePoints',ascending=False).reset_index(drop=True)
    ironLadder.to_csv('ironLadder.csv',index=False)
    print('Not found')

In [6]:
def getMatch(matchID):
    root = 'https://americas.api.riotgames.com/lol/match/v5/matches/'
    middle = '?api_key='
    api_url = root + matchID + middle + api_key
    time.sleep(1)
    result = requests.get(api_url)
    if result.status_code != 200:
        raise Exception("Status Code:",result.status_code)
    
    return result.json()

In [7]:
def getListMatches(puuid,start,count):
    root = 'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/'
    middle = '/ids?type=ranked&'
    part = 'start='+str(start)+'&count='+str(count)
    api_url = root +str(puuid)+middle + part + '&api_key=' + api_key
    page = requests.get(api_url).json()
    return page 

In [8]:
def getKDA(information,puuid):
    kda = 0
    try:
        participant = information['metadata']['participants'].index(puuid)
        kills = information['info']['participants'][participant]['kills']
        deaths = information['info']['participants'][participant]['deaths']
        assists = information['info']['participants'][participant]['assists']
        #print(information['info']['participants'][participant]['championName'])
        return (kills+assists)/max(deaths,1)
    except Exception as e:
        print(information)
        print("Exception",e)


In [9]:
def getCS(information,puuid):
    participant = information['metadata']['participants'].index(puuid)
    cs = information['info']['participants'][participant]['totalMinionsKilled']
    minutes = int(information['info']['gameDuration'])/60
    #print(cs/minutes)
    return cs/minutes

In [10]:
print(len(ironLadder))

16462


In [ ]:
#First get 3000 matches
#For each match, find the KDA of each person 

In [12]:
# For each person in the ladder, look at their 10-20 games and get their average cs per min, KDA
average_iron_1 = pd.DataFrame({'puuid':[],'KDA':[],'CS/Min':[]})
for i in ironLadder.itertuples():
    time.sleep(10)
    puuid = get_puuid(i.summonerId)
    matches = getListMatches(puuid=puuid,start = 10,count = 10)
    kda_mean = 0 
    cs_mean = 0 
    for m in matches:
        information = getMatch(m)
        kda = getKDA(information,puuid)
        cs = getCS(information,puuid)
        kda_mean += kda
        cs_mean += cs
    kda_mean= kda_mean/len(matches)
    cs_mean = cs_mean/len(matches)
    new_row = pd.DataFrame({'puuid':[puuid],'KDA':[kda_mean],'CS/Min':[cs_mean]})
    average_iron_1 = pd.concat([average_iron_1,new_row],ignore_index=True)
    if len(average_iron_1) % 10 == 0:
        print(len(average_iron_1))
        time.sleep(120)


10
20
30
40


KeyboardInterrupt: 

In [ ]:
ironLadder['summonerId'] = ironLadder['summonerId'].map(average_iron_1)